# Basic CV model trained on CIFAR10 (W&B)

## Setup

In [ ]:
%pip install -r requirements.txt

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb

## W&B config

In [ ]:
wandb.login()

Using W&B Sweeps to run hyperparemter optimisation experiments

In [3]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    }
}

In [4]:
parameters = {
    # fixed
    'batch_size': {
        'value': 2
    },
    'momentum': {
        'value': .9
    },

    # variable
    'epochs': {
        'values': [1, 2, 4, 8],
    },
    'lr': {
        'values': [0.00001, .0001, .001],
    },
    'optimizer': {
        'values': ['adam', 'sgd'],
    },
    'fc_layer_1_size': {
        'values': [60, 120, 180],
    },
    'fc_layer_2_size': {
        'values': [42, 84, 126],
    },
    'dropout': {
        'values': [0, .2, .4],
    }
}
sweep_config['parameters'] = parameters

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='wandb201')

## Utils

In [6]:
class Net(nn.Module):
    def __init__(self, fc_layer_1_size, fc_layer_2_size):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, fc_layer_1_size)
        self.fc2 = nn.Linear(fc_layer_1_size, fc_layer_2_size)
        self.fc3 = nn.Linear(fc_layer_2_size, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
def get_predictions(model, inputs, optimizer):
    optimizer.zero_grad()
    return model(inputs)

def update_model(model, data, loss_fn, optimizer):
    inputs, labels = data
    predictions = get_predictions(model, inputs, optimizer)
    loss = loss_fn(predictions, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def get_transforms(mean, std):
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

def fetch_data(transform):
    return torchvision.datasets.CIFAR10(
        root='./data',
        train=True,
        download=True,
        transform=transform,
    )

def get_data_wo_transform():
    transform = transforms.Compose([transforms.ToTensor()])
    return fetch_data(transform)

def get_data(transforms, batch_size=4):
    trainset = fetch_data(transforms)
    return torch.utils.data.DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

def calculate_mean_std(dataset):
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=64,
        shuffle=False,
        num_workers=2
    )

    channels_sum, channels_squared_sum, num_batches = 0, 0, 0

    for data, _ in dataloader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

def save_model(model, path):
    torch.save(
        model.state_dict(),
        path
    )

Configurable hyperparameters

In [7]:
def build_optimizer(model, optimizer, learning_rate, momentum):
    if optimizer == "sgd":
        optimizer = optim.SGD(
            model.parameters(),
            lr=learning_rate,
            momentum=momentum
        )
    elif optimizer == "adam":
        optimizer = optim.Adam(
            model.parameters(),
            lr=learning_rate
        )
    return optimizer

## Run

In [8]:
def train():
    # create a W&B run w/ `wandb.init()`
    with wandb.init(project='wandb201', config=None):
        config = wandb.config
        
        # setup
        mean, std = calculate_mean_std(get_data_wo_transform())
        transforms = get_transforms(mean, std)
        data = get_data(transforms, config.batch_size)
        model = Net(
            fc_layer_1_size=config.fc_layer_1_size,
            fc_layer_2_size=config.fc_layer_2_size,
        )
        loss_fn = nn.CrossEntropyLoss()
        optimizer = build_optimizer(
            model,
            config.optimizer,
            config.lr,
            config.momentum,
        )

        print('starting training run...')
        # training
        for epoch in range(config.epochs):
            print(f'epoch: {epoch}')
            for i, batch in enumerate(data):
                loss = update_model(
                    model,
                    batch,
                    loss_fn,
                    optimizer,
                )
                wandb.log({'epoch': epoch, 'loss': loss})
        print('finished training run')
        path = './cifar_net.pth'
        save_model(model, path)

In [ ]:
wandb.agent(sweep_id, train, project='wandb201')